In [1]:
from gensim.models import KeyedVectors
import numpy as np
import matplotlib.pyplot as plt
from os import sep

%matplotlib inline

embeddings_path = "embeddings_wiki2017"
dataset_path = "data"

### Récupérer des word embeddings 

Ce word embedding a les caractéristiques suivantes:

- Gensim Continuous Skipgram
- taille de vecteur ${300}$
- window ${5}$
- entrainé sur wikipédia février 2017 en langue anglaise
- pas de lemmatisation
- ${302866}$ mots

In [2]:
model = KeyedVectors.load_word2vec_format(embeddings_path + sep + "model.bin", binary=True)

In [3]:
vocabulary = [w for w in model.vocab]

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', vocabulary=vocabulary, binary=True)

### On Récupère les paires de pertinence/non pertinence pour chaque requête 

On génère un dictionnaire qui contient pour chaque requête en clé, un dictionnaire contenant 2 listes:
- "relevant" contenant des id de document pertinents pour la requête.
- "irrelevant" contenant des id de document non pertinents pour la requête.

In [5]:
paires = {}

with open(dataset_path + sep + "qrels.robust2004.txt", "r") as f:
    for line in f:
        lol = line.split()
        paires.setdefault(lol[0], {})
        paires[lol[0]].setdefault('relevant', []) 
        paires[lol[0]].setdefault('irrelevant', [])
        if lol[-1] == '1':
            paires[lol[0]]["relevant"].append(lol[2])
        else:
            paires[lol[0]]["irrelevant"].append(lol[2])

### On récupère les requêtes:

Elles se trouvent sous forme de tuple ([mots clés], [texte de la requête]). On ne garde que les mots clés.

In [10]:
import ast

with open(dataset_path + sep + "robust2004.txt", "r") as f:
    queries = ast.literal_eval(f.read())
queries = {d:queries[d][0] for d in queries}

In [13]:
print(queries["301"])
print(queries["302"])

international organized crime
poliomyelitis and post polio


### On peut maintenant construire l'histogramme des interactions entre les embeddings de la requête et ceux du document.

On prend comme exemple 4 bins: ${[-1, -0.5]}$ ${[-0.5, 0]}$ ${[0, 0.5]}$ ${[0.5, 1]}$

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

In [56]:
intvls = np.linspace(-1, 1, 5)

def hist(x,y,intervals=intvls):
    histo = []
    for i in x.nonzero()[1]:
        for j in y.nonzero()[1]:
            histo.append(cosine_similarity([model.vectors[i]], [model.vectors[j]])[0][0])
    histo = np.histogram(histo, bins=intervals)
    print(histo[0])

In [46]:
x = vectorizer.transform([queries["301"]])
y = vectorizer.transform(["I am a criminal and i kill"]) #rajouter prétraitements
y2 = vectorizer.transform(["I am a peaceful dude"])

In [57]:
hist(x,y)

[0 0 8 1]


In [58]:
hist(x,y2)

[0 0 6 0]
